## Plot WROMY  with Rotation / Tilt

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from numpy import nan
from pathlib import Path
from obspy import UTCDateTime, Stream

from andbro__get_timeaxis import __get_timeaxis

In [2]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
    bay_path = '/home/andbro/bay200/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/bay200/'
elif os.uname().nodename == 'lin-ffb-01':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/bay200/'

### Configurations

In [131]:
config = {}

config['channel'] = None

config['tbeg'] = UTCDateTime("2022-03-27 00:00")
config['tend'] = UTCDateTime("2022-03-30 00:00")

config['path_to_sds'] = archive_path+"romy_archive/"

config['path_to_figs'] = data_path+"wromy/figures/"

config['resample'] = 20

config['colors'] = {  'WS1':'darkgreen', 
                      'WS4':'purple', 
                      'WS5':'darkred',
                      'WS6':'darkblue',
                      'WS7':'darkorange',
                      'WS8':'darkcyan',
                      'WS9':'cyan',
                     }

### Methods

In [4]:
def __indicate_gaps_with_nan(df, config):

    differences = np.diff(df.totalSeconds, n=1)


    ## ______________

    sample_time_errors = [j for j in differences if j != config['resample']]

    if len(sample_time_errors) != 0:
        print(f"  -> ERROR: Found {len(sample_time_errors)} errors for the sampling time!\n")


    ## ______________

    gaps = [list(differences).index(k) for k in differences if k > 2*config['resample']] or []
    if gaps and gaps[0] in [0, 0.0]:
        gaps.pop(0)
    del differences

    for x in gaps:
        fill_row = [i+config['resample'] if n not in [3,4,5] else np.nan for n, i in enumerate(df.iloc[x,:])]
        fill_row[0] = int(df.iloc[x,0])
        fill_row[1] = int(df.iloc[x,1])
        fill_row[2] = int(df.iloc[x,2])
        df.loc[x+0.5] = fill_row


    df = df.sort_index().reset_index(drop=True).convert_dtypes()

    print(f"  -> Marked {len(gaps)} gaps with NaN values!\n")

    return df

## Load FURT data

In [87]:
from andbro__load_FURT_stream import __load_furt_stream

furt = __load_furt_stream(config['tbeg'], config['tend'], path_to_archive=bay_path+"gif_online/FURT/WETTER/")

/home/andbro/andbro_python/andbro__load_FURT_stream.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = to_datetime(df['datetime'], format="%d%m%y %H%M%S", errors="ignore")


### Load WROMY data

In [89]:
from andbro__load_WROMY_stream import __load_wromy_stream

ws = Stream()

for i in [1,4,5,6,7,8,9]:

    ws += __load_wromy_stream(config['path_to_sds'], f"BW.WROMY..WS{i}", config['tbeg'], config['tend'])

Specified end: 2022-03-30T00:00:00.000000Z 
Trace end:     2022-03-29T23:59:59.000000Z
Specified end: 2022-03-30T00:00:00.000000Z 
Trace end:     2022-03-29T23:59:59.000000Z
Specified end: 2022-03-30T00:00:00.000000Z 
Trace end:     2022-03-29T23:59:59.000000Z
Specified end: 2022-03-30T00:00:00.000000Z 
Trace end:     2022-03-29T23:59:59.000000Z
Specified end: 2022-03-30T00:00:00.000000Z 
Trace end:     2022-03-29T23:59:59.000000Z
Specified end: 2022-03-30T00:00:00.000000Z 
Trace end:     2022-03-29T23:59:59.000000Z
Specified end: 2022-03-30T00:00:00.000000Z 
Trace end:     2022-03-29T23:59:59.000000Z


## Load Tilt

In [90]:
from andbro__readYaml import __readYaml
from andbro__read_sds import __read_sds

## tiltmeter configurations
confTilt = __readYaml(f"{root_path}Documents/ROMY/tiltmeter/", "tiltmeter.conf")


# ROMYT0 = __read_sds(config['path_to_sds'], "BW.ROMYT..MA*", config['tbeg'], config['tend'])
# ROMYT0.sort();
# ROMYT0.resample(1.0, no_filter=False)
# ROMYT0.merge();


# TROMY0 = __read_sds(config['path_to_sds'], "BW.TROMY..MA*", config['tbeg'], config['tend'])
# TROMY0.sort();
# TROMY0.resample(1.0, no_filter=False);
# TROMY0.merge();


BROMY0 = __read_sds(config['path_to_sds'], "BW.DROMY..LA*", config['tbeg'], config['tend'])
BROMY0.sort()
BROMY0.merge()

3 Trace(s) in Stream:
BW.DROMY..LAE | 2022-03-27T00:00:00.000000Z - 2022-03-30T00:00:00.000000Z | 1.0 Hz, 259201 samples
BW.DROMY..LAN | 2022-03-27T00:00:00.000000Z - 2022-03-30T00:00:00.000000Z | 1.0 Hz, 259201 samples
BW.DROMY..LAT | 2022-03-27T00:00:00.000000Z - 2022-03-30T00:00:00.000000Z | 1.0 Hz, 259201 samples

In [123]:
def __conversion_to_tilt(st, conf):

    st0 = st.copy()

    def convertTemp(trace):
        Tvolt = trace.data * conf.get('gainTemp')
        coeff = conf.get('calcTempCoefficients')
        return coeff[0] + coeff[1]*Tvolt + coeff[2]*Tvolt**2 + coeff[3]*Tvolt**3

    def convertTilt(trace, conversion, sensitivity):
        return trace.data * conversion * sensitivity

    for tr in st0:
        if tr.stats.channel[-1] == 'T':
            tr.data = convertTemp(tr)
        elif tr.stats.channel[-1] == 'N':
            tr.data = convertTilt(tr, conf['convTN'], conf['gainTilt'])
        elif tr.stats.channel[-1] == 'E':
            tr.data = convertTilt(tr, conf['convTE'], conf['gainTilt'])
        else:
            print("no match")

    print(f"  -> converted data of {st[0].stats.station}")
    return st0

In [125]:
BROMY = __conversion_to_tilt(BROMY0, confTilt['BROMY'])

  -> converted data of DROMY


## Plot WROMY and Tilt

In [132]:
def __makeplot_WROMY_FURT_TILT():

    from numpy import mean

    Nrow, Ncol = 4, 1

    font = 12

    tilt_scaling = 1e6
    time_scaling = 3600

    fig, ax = plt.subplots(Nrow, Ncol, figsize=(15, 8), sharex=True)

    for _i in [1,4,5,6,7,8,9]:

        ax[0].plot(ws.select(location=f"*{_i}", channel="*T")[0].times()/time_scaling, ws.select(location=f"*{_i}", channel="*T")[0].data,
                   color=config['colors'][f'WS{_i}'], alpha=0.7, label=f"WS{_i}")
        ax[1].plot(ws.select(location=f"*{_i}", channel="*P")[0].times()/time_scaling, ws.select(location=f"*{_i}", channel="*P")[0].data,
                   color=config['colors'][f'WS{_i}'], alpha=0.7)
        ax[2].plot(ws.select(location=f"*{_i}", channel="*H")[0].times()/time_scaling, ws.select(location=f"*{_i}", channel="*H")[0].data,
                   color=config['colors'][f'WS{_i}'], alpha=0.7)

    ax[0].plot(furt[0].times()/time_scaling, furt.select(channel="*T")[0].data, color="lightgrey", zorder=0, label="FURT")
    ax[1].plot(furt[0].times()/time_scaling, furt.select(channel="*P")[0].data, color="lightgrey", zorder=0)
    ax[2].plot(furt[0].times()/time_scaling, furt.select(channel="*H")[0].data, color="lightgrey", zorder=0)

    tilt_N = ( BROMY.select(channel="*N")[0].data - mean(BROMY.select(channel="*N")[0].data[:100]) ) * tilt_scaling
    tilt_E = ( BROMY.select(channel="*E")[0].data - mean(BROMY.select(channel="*E")[0].data[:100]) ) * tilt_scaling

    ax[3].plot(BROMY.select(channel="*N")[0].times()/time_scaling, tilt_N, label="N-S")
    ax[3].plot(BROMY.select(channel="*E")[0].times()/time_scaling, tilt_E, label="E-W")

    ax[3].legend(loc=1)

    ax[0].legend(loc='upper center', ncol=7+1, bbox_to_anchor=(0.5, 1.3), fancybox=True, fontsize=font-2)

    for _n in range(Nrow):
        ax[_n].grid(ls=":", zorder=0)
        ax[_n].set_xlim(left=0)

    ax[0].set_ylabel("Temperature (°C)",fontsize=font)
    ax[1].set_ylabel("Air Pressure (hPa)",fontsize=font)
    ax[2].set_ylabel("rel. Humidity (%)",fontsize=font)
    ax[3].set_ylabel("Tilt ($\mu$rad)", fontsize=font)
    ax[3].set_xlabel("Time (hours)", fontsize=font)

    plt.show();
    return fig

In [ ]:
fig = __makeplot_WROMY_FURT_TILT()

fig.savefig(config['path_to_figs']+f"WROMY_Tilt_effected_by_temperature.png", format="png", dpi=150, bbox_inches='tight')

## WROMY & Rotation

In [ ]:
config_rot = {}

config_rot['seed'] = "BW.ROMY.10.BJZ"

config_rot['tbeg'] = config['tbeg']
config_rot['tend'] = config['tend']-300

config_rot['repo'] = "george"

In [ ]:
st00, inv = __querrySeismoData(
                            seed_id=config_rot.get("seed"),
                            starttime=config_rot.get("tbeg"),
                            endtime=config_rot.get("tend"),
                            repository=config_rot.get("repo"),
                            path=None,
                            restitute=False,
                            detail=False,
                            fill_value=-9999,
                            )

In [ ]:
# st = st00.copy()

# st.detrend('linear')

# st.remove_response(inv, output="VEL", zero_mean=True)

# st.taper(0.1)

# fmin, fmax = 0.01, 5.0
# # st.filter('bandpass', freqmin=fmin, freqmax=fmax, corners=4, zerophase=True)
# st.filter('lowpass', freq=fmax, corners=8, zerophase=True)

# st.resample(fmax*2)

# st.plot(equal_scale=False);

In [ ]:
def __make_plot_all_stations_and_furt_and_rotation(data, furt, rota, config, events=None):
    
    N = 4
    font = 13
    datasize = 0
    
    fig, axes = plt.subplots(N,1, figsize=[15,13], sharex=True)
    
    plt.subplots_adjust(hspace=0.1)
    
    max_val, min_val = np.zeros(N)*np.nan, np.zeros(N)*np.nan
    
    timeaxis_furt, ticks_furt, ticklabels_furt, text_furt = __get_timeaxis(
                                                                           dates=furt.iloc[:,0], 
                                                                           times=furt.iloc[:,1], 
                                                                           unit="date", 
                                                                           unitmode="absolute",
                                                                           dateformat="ddmmyy",
                                                                          ) 

    timeaxis_rota, ticks_rota, ticklabels_rota, text_rota = __get_timeaxis(
                                                                           streamstats=rota[0].stats,
                                                                           unit="date", 
                                                                           unitmode="absolute",
                                                                          )   
    NN = 0
    for station in data.keys():
        
        df = data.get(station)

        for u in range(3):
            maximum = df.iloc[:,u+3].dropna().max()
            minimum = df.iloc[:,u+3].dropna().min()
            if maximum > max_val[u] or np.isnan(max_val[u]):
                max_val[u] = maximum
            if minimum < min_val[u] or np.isnan(min_val[u]):
                min_val[u] = minimum
            
        
        if NN != len(df.iloc[:,1]): 
            timeaxis, ticks, ticklabels, text = __get_timeaxis(dates=df.iloc[:,1], 
                                                               times=df.iloc[:,2], 
                                                               unit="date", 
                                                               unitmode="absolute",
                                                               dateformat="yyyymmdd",
                                                              )
            
        NN = len(df.iloc[:,1])
        
        ## select ticks and ticklabels for longest data series
        if df.shape[0] > datasize:
            datasize = df.shape[0]
            xticks = ticks
            xlabels = ticklabels
            timeaxis_min, timeaxis_max = timeaxis[0], timeaxis[-1]


        ## plot data and adjust axes automatically
        for i in range(N):
            
            if i == N-1:
                axes[i].plot(timeaxis_rota, rota.select(channel="*Z")[0].data, color='k', lw=1.5, zorder=2, label="TROMY E")
                axes[i].set_ylim(-1e-7, 1e-7)
            else:
                axes[i].plot(timeaxis, df.iloc[:,i+3].rolling(60).mean(), color=config['colors'][station], lw=1.5, zorder=2, label=station)

                if station == list(data.keys())[-1]:
                    axes[i].plot(timeaxis_furt, furt.iloc[:,i+2], color='darkgrey', lw=1.5, zorder=1, label="FURT")

            axes[i].grid(ls="--",color='grey', zorder=0)
            
            if i == 0:
                axes[i].set_ylabel("Temperature (°C)",fontsize=font)
            elif i == 1:
                axes[i].set_ylabel("Air Pressure (hPa)",fontsize=font)
            elif i == 2:
                axes[i].set_ylabel("rel. Humidity (%)",fontsize=font)
                axes[i].set_xlim(timeaxis_min, timeaxis_max)
            elif i == 3:
                axes[i].set_ylabel("Rotation (rad/s)", fontsize=font)
                
                
#             axes[N-1].set_xticklabels(xlabels[::4], fontsize=font-2)
#             axes[N-1].set_xticks(xticks[::4])
            axes[N-1].set_xticks(xticks)
            axes[N-1].set_xticklabels(xlabels, fontsize=font-2)
            axes[N-1].set_xlabel(text, fontsize=font)
            axes[N-2].legend(loc='upper center', ncol=7+1, bbox_to_anchor=(0.5, 3.4), fancybox=True, fontsize=font-2)

            
            if events:
                for event in events:
#                     axes[i].axvline(event, color='r', zorder=0, ls="-.")
                    axes[i].axvspan(event[0], event[1], color="lightgrey", alpha=0.4, zorder=1)
          
        ## adjust y-limits
#     for k in range(N):
#         axes[k].set_ylim(min_val[k]-0.005*min_val[k], max_val[k]+0.005*min_val[k])
        
    plt.show();
    return fig

__make_plot_all_stations_and_furt_and_rotation(data, furt, st, config, events=None);